In [ ]:
# remove ''s from text
# loop through tweets
# add positive/negative rules
# target not in a url?

In [98]:
import pandas as pd
import re
import math

posRules = pd.read_csv("sentimentRulesPos.csv")
negRules = pd.read_csv("sentimentRulesNeg.csv")
def getPosSentiment(target,text):
    for i, rule in posRules.iterrows():
        rulePass = False
        
        if(isinstance(rule['include'], str)):
            includes = re.findall('"([^"]*)"', rule['include'])
            for inc in includes:
                #print(inc)
                adj = inc.replace('"','').replace('?','.').replace('*','.*?').replace('XXX',target)
                #print(adj)
                if(re.search(adj,text)):
                    rulePass = True

        #print(rulePass)
        if(isinstance(rule['not'], str)):
            notInc = re.findall('"([^"]*)"', rule['not'])
            for notI in notInc:
                #print(rulePass)
                #print(notI)
                adj = notI.replace('"','').replace('?','.').replace('*','.*?').replace('XXX',target)
                #print(adj)
                if(re.search(adj,text)):
                    rulePass = False
        
        if(rulePass):
            return True
    return False

def getNegSentiment(target,text):
    for i, rule in negRules.iterrows():
        rulePass = False
        
        if(isinstance(rule['include'], str)):
            includes = re.findall('"([^"]*)"', rule['include'])
            for inc in includes:
                #print(inc)
                adj = inc.replace('"','').replace('?','.').replace('*','.*?').replace('XXX',target)
                #print(adj)
                if(re.search(adj,text)):
                    rulePass = True

        #print(rulePass)
        if(isinstance(rule['not'], str)):
            notInc = re.findall('"([^"]*)"', rule['not'])
            for notI in notInc:
                #print(rulePass)
                #print(notI)
                adj = notI.replace('"','').replace('?','.').replace('*','.*?').replace('XXX',target)
                #print(adj)
                if(re.search(adj,text)):
                    rulePass = False
        
        if(rulePass):
            return True
    return False

In [43]:
queryTarget = 'MSFT'

In [12]:
import requests 

consumer_key = '5CIKmLJlsHGlFJmembtiWLpaF'
consumer_secret = 'TN0nzhbHNWSVEkg2kAkOiPkExFgcuWNGs87nXnzYQM8HhGd3cu'
access_token_key = '30576108-ovRUQqF5EpSyra6yUk7dlmsDACiX8Zin56lWiI78c'
access_token_secret = 'ETy2w1WPYr4v2re3ztS2zLsxFR2Eua7TASygByViefjZe'
            
from requests_oauthlib import OAuth1
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
auth = OAuth1(consumer_key, consumer_secret, access_token_key, access_token_secret)
requests.get(url, auth=auth)

r = requests.get(
    'https://api.twitter.com/1.1/tweets/search/30day/development.json?'+
    'query='+queryTarget+'&toDate=201903260000&&fromDate=201903010000&maxResults=10', 
    auth=auth)

In [99]:
import time
from datetime import datetime

tweets = []
for tweet in r.json()['results']:
    curtext = ''
    if('retweeted_status' in tweet.keys()):
        if('extended_tweet' in tweet['retweeted_status'].keys()):
            curtext = tweet['retweeted_status']['extended_tweet']['full_text']
        else:
            curtext = tweet['retweeted_status']['text']
    else:
        curtext = tweet['extended_tweet']['full_text']
        
    posSent = getPosSentiment(queryTarget,curtext)
    negSent = getNegSentiment(queryTarget,curtext)
    tt = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
    ut = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
    curtweet = {"tid":tweet['id'],'text':curtext,'quote_count':tweet['quote_count'],'reply_count':tweet['reply_count'],
               'retweet_count':tweet['retweet_count'],'favorite_count':tweet['favorite_count'],'lang':tweet['lang'],
               'userid':tweet['user']['id'],'tweet_date':tt,'username':tweet['user']['screen_name'],
                'followers_count':tweet['user']['followers_count'],'friends_count':tweet['user']['friends_count'],
                'listed_count':tweet['user']['listed_count'],'favourites':tweet['user']['favourites_count'],
                'statuses_count':tweet['user']['statuses_count'],'created_at':ut,
                'langu':tweet['user']['lang'],'pos':posSent,'neg':negSent,'query':queryTarget}
    tweets.append(curtweet)

tweets = tuple(tweets)

In [101]:
import psycopg2
import traceback

try:
    conn=psycopg2.connect("dbname='temp' user='postgres' password='TestPass'")
    conn.autocommit = True
except:
    print("I am unable to connect to the database.")
    
cur = conn.cursor()
try:
    cur.executemany("""INSERT INTO public."TwitterUser"(id, name, followers_count, friends_count, 
listed_count, favourites_count, statuses_count, created_at, lang) VALUES (%(userid)s, 
%(username)s, %(followers_count)s, %(friends_count)s, %(listed_count)s, %(favourites)s, 
%(statuses_count)s, %(created_at)s, %(langu)s)
ON CONFLICT DO NOTHING;""",tweets)
    
    cur.executemany("""INSERT INTO public."Tweet"(
id, text, quote_count, reply_count, retweet_count, favorite_count, lang, "userId", created_at, query, positive, negative)
VALUES (%(tid)s, %(text)s, %(quote_count)s, %(reply_count)s, %(retweet_count)s, %(favorite_count)s, %(lang)s, 
%(userid)s, %(tweet_date)s, %(query)s, %(pos)s, %(neg)s);""",tweets)
    conn.commit()
    
except Exception:
    traceback.print_exc()
    print("I can't add the test data!")

cur.close()
conn.close()